# Portfolio B: Topic Discovery
**Map the conversation landscape of an AI agent social network**

Moltbook is a leaked social network where AI agents post about everything from philosophy to memes. Your mission: use topic modeling to discover *what* agents talk about, *how* topics relate to each other, and whether the ground-truth labels capture the full picture.

**Dataset**: Moltbook (44K AI agent posts, 9 ground-truth categories)  
**Your goal**: Discover topics with BERTopic, name them with an LLM, and compare to ground-truth labels.

### Deliverables
- BERTopic model with tuned parameters
- LLM-generated topic names
- Visualization: topic map + at least one other
- Cross-tabulation: discovered topics vs ground-truth labels
- Brief model card

**Estimated time**: Sprint 1 (55 min) + Sprint 2 (90 min)

## Setup

In [ ]:
!pip install -q datasets sentence-transformers bertopic umap-learn hdbscan openai scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from sentence_transformers import SentenceTransformer

## 1. Load & Explore

In [ ]:
from datasets import load_dataset

dataset = load_dataset("TrustAIRLab/Moltbook", "posts", split="train")
df = dataset.to_pandas()

df["title"] = df["post"].apply(lambda x: x.get("title", "") if isinstance(x, dict) else "")
df["content"] = df["post"].apply(lambda x: x.get("content", "") if isinstance(x, dict) else "")
df["title"] = df["title"].fillna("").astype(str)
df["content"] = df["content"].fillna("").astype(str)
df["text"] = (df["title"].str.strip() + " . " + df["content"].str.strip()).str.strip()
df = df[df["text"].str.len() > 10].reset_index(drop=True)

print(f"Total posts: {len(df)}")
print(f"\nGround-truth categories:")
print(df["topic_label"].value_counts())

In [ ]:
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"https?://\S+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["text_clean"] = df["text"].apply(clean_text)
df = df[df["text_clean"].str.len() > 3].reset_index(drop=True)

# Subsample for speed — increase if you have time
df_sample = df.sample(5000, random_state=42).reset_index(drop=True)
print(f"Working with {len(df_sample)} posts")

## 2. Encode Texts

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(df_sample["text_clean"].tolist(), show_progress_bar=True, batch_size=64)
print(f"Embeddings: {embeddings.shape}")

## 3. Baseline BERTopic Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

stopwords = list(ENGLISH_STOP_WORDS) + [
    "just", "like", "really", "think", "know", "want",
    "got", "get", "one", "would", "could", "also",
    "ai", "agent", "agents", "moltbook", "post", "posts",
    "bot", "bots", "human", "humans",
]

topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=CountVectorizer(stop_words=stopwords, ngram_range=(1, 2), min_df=5),
    umap_model=UMAP(n_neighbors=15, n_components=5, metric="cosine", random_state=42),
    hdbscan_model=HDBSCAN(min_cluster_size=50, min_samples=10, prediction_data=True),
    representation_model=KeyBERTInspired(),
    verbose=True,
)

topics, probs = topic_model.fit_transform(df_sample["text_clean"].tolist(), embeddings=embeddings)
print(f"\nTopics found: {len(set(topics)) - 1}")
print(f"Outliers: {sum(t == -1 for t in topics)}")

In [ ]:
topic_model.get_topic_info().head(15)

In [ ]:
topic_model.visualize_barchart(top_n_topics=12, n_words=8)

## 4. Your Turn: Tune & Improve

Try changing these parameters and see how the topics change:
- `min_cluster_size`: smaller = more topics, larger = fewer broad topics
- `n_neighbors`: affects how local/global the UMAP structure is
- `min_df` in CountVectorizer: filter rare words

In [ ]:
# YOUR CODE HERE — Try different parameters
# topic_model_v2 = BERTopic(
#     ...
# )

## 5. Your Turn: LLM Topic Naming
Use Groq to give topics human-readable names (see NB04 for the full pattern).

In [ ]:
# YOUR CODE HERE — LLM topic naming via Groq
# import openai
# from bertopic.representation import OpenAI as OpenAIRepresentation
#
# GROQ_API_KEY = ""  # @param {type:"string"}
# groq_client = openai.OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")
# ...

## 6. Compare to Ground Truth

In [ ]:
df_sample["topic"] = topics
df_assigned = df_sample[df_sample["topic"] != -1].copy()
top_topics = df_assigned["topic"].value_counts().head(10).index.tolist()
df_top = df_assigned[df_assigned["topic"].isin(top_topics)]

ct = pd.crosstab(df_top["topic_label"], df_top["topic"], margins=True)
print(ct.to_string())

## 7. Visualize

In [ ]:
# Topic map
topic_model.visualize_topics()

In [ ]:
# YOUR CODE HERE — try visualize_documents(), visualize_hierarchy(), or visualize_heatmap()

## 8. Model Card

| Field | Value |
|-------|-------|
| **Task** | Topic discovery on AI agent posts |
| **Dataset** | Moltbook (N=5000 sample) |
| **Topics found** | _number_ |
| **Outlier rate** | _percentage_ |
| **Best insight** | _what did you discover?_ |
| **Ground-truth match** | _do topics align with the 9 labels?_ |
| **Weakness** | _what topics are confused or missing?_ |
| **Improvement idea** | _what you'd try next_ |